<a href="https://colab.research.google.com/github/cubiwan/colabs_IA/blob/main/Entrena_tu_SLM_sin_tener_ni_idea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cómo entrenar tu propio modelo pequeño de lenguaje sin tener ni idea de lo que estas haciendo

## **1**
Instalamos [llama.cpp](https://github.com/ggerganov/llama.cpp) y lo compilamos (si lo tienes instalado en local puedes pasar a los siguientes pasos)

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp
!make

## **2.a**
Si quieres entrenar un modelo inspirado en Shakespeare sigue este camino

In [ ]:
#Descargamos el dataset
!wget https://raw.githubusercontent.com/brunoklein99/deep-learning-notes/master/shakespeare.txt

#Entrenamos el modelo
!./train-text-from-scratch --vocab-model ./models/ggml-vocab-llama.gguf --ctx 64 --embd 256 --head 8 --layer 16 --checkpoint-in chk-shakespeare-256x16.gguf --checkpoint-out chk-shakespeare-256x16.gguf --model-out ggml-shakespeare-256x16-f32.gguf --train-data "shakespeare.txt" -b 16 --seed 1 --adam-iter 256 --no-checkpointing

## **2.b**
Si quieres entrenar un modelo inspirado en El Quijote sigue este camino

In [ ]:
#descargamos el dataset
!wget https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt

Limpiamos el texto

In [ ]:
# Nombre del archivo de entrada y salida
nombre_archivo_entrada = "el_quijote.txt"
nombre_archivo_salida = "el_quijote_clean.txt"

# Función para reemplazar caracteres especiales
def reemplazar_caracteres_especiales(texto):
    # Mapeo de caracteres especiales a sus equivalentes sin tilde/diéresis
    reemplazos = {
        "á": "a",
        "é": "e",
        "í": "i",
        "ó": "o",
        "ú": "u",
        "ü": "u",
        "Á": "A",
        "É": "E",
        "Í": "I",
        "Ó": "O",
        "Ú": "U",
        "Ü": "U",
        "ñ": "n",
        "Ñ": "N"
    }

    # Realizar los reemplazos
    for clave, valor in reemplazos.items():
        texto = texto.replace(clave, valor)

    return texto

try:
    # Abrir el archivo de entrada para lectura
    with open(nombre_archivo_entrada, "r", encoding="utf-8") as archivo_entrada:
        # Leer el contenido del archivo
        contenido = archivo_entrada.read()

    # Reemplazar caracteres especiales
    contenido_modificado = reemplazar_caracteres_especiales(contenido)

    # Abrir el archivo de salida para escritura
    with open(nombre_archivo_salida, "w", encoding="utf-8") as archivo_salida:
        # Escribir el contenido modificado en el archivo de salida
        archivo_salida.write(contenido_modificado)

    print("Reemplazo completado con éxito. El archivo de salida se ha creado como", nombre_archivo_salida)

except FileNotFoundError:
    print(f"No se pudo encontrar el archivo de entrada: {nombre_archivo_entrada}")
except Exception as e:
    print("Se produjo un error:", str(e))

In [ ]:
#entrenamos el modelo con el dataset limpio
!./train-text-from-scratch --vocab-model ./models/ggml-vocab-llama.gguf --ctx 64 --embd 256 --head 8 --layer 16 --checkpoint-in chk-quijote-256x16.gguf --checkpoint-out chk-quijote-256x16.gguf --model-out ggml-quijote-256x16-f32.gguf --train-data "el_quijote_clean.txt" -b 16 --seed 1 --adam-iter 256 --no-checkpointing

## **2.c**
Si quieres entrenar un modelo con datos estructurados sigue este camino

Primero generamos los datos

In [ ]:
import random

# Define las estructuras de datos
agente = ["oye", "hey", "ia", "bot", "eh", "escucha", "", "", "", "", "", "", ""]

accion = [
    ["prende", "enciende", "pon", "activa", "ilumina", "conecta", "enchufa", "on"],
    ["apaga", "quita", "desconecta", "desenchufa", "off"]
]

det1 = ["el", "la", ""]

luz = ["luz", "bombilla", "iluminacion", "led"]
aire = ["aire", "ac", "acondicionador", "aire acondicionado", "frio"]

det2 = ["del", "de la", "de", "la", "el", ""]

lugar = [
    ["bano", "lavabo", "cuarto de bano", "sanitario", "wc"],
    ["dormitorio", "cama"],
    ["entrada", "recibidor", "pasillo", "recibidor"],
    ["cocina", "fogon", "fogones"],
    ["salon", "comedor"],
    ["despacho", "oficina"],
    ["casa", "hogar", ""]
]

det3 = ["la","lo", "le", "", ""]

comando_accion = ["ON", "OFF"]
comando_lugar = ["LUZ_BN", "LUZ_DR", "LUZ_EN", "LUZ_CO", "LUZ_SL", "LUZ_DP", "AC"]


def crea_sentencia():
  # Elegir una acción (0 o 1) y un lugar (0...6) al azar
  indice_accion = random.randint(0, 1)
  indice_lugar = random.randint(0, 6)

  # Elegir una acción y un lugar basados en los índices
  accion_elegida = random.choice(accion[indice_accion])
  lugar_elegido = random.choice(lugar[indice_lugar])

  # Si el indice_lugar lugar es 6 (casa), elegir una palabra del array "aire"
  if indice_lugar == 6:
      aparato_elegido = random.choice(aire)
  else:
      aparato_elegido = random.choice(luz)

  # Construir la oración
  if random.randint(0, 1) == 1:
    orden = f"{random.choice(agente)} {accion_elegida} {random.choice(det1)} {aparato_elegido} {random.choice(det2)} {lugar_elegido}"
  else:
    orden = f"{random.choice(agente)} {random.choice(det1)} {aparato_elegido} {random.choice(det2)} {lugar_elegido} {accion_elegida}{random.choice(det3)}"

  # Construir la sentencia
  sentencia = f"<s>\n [ORDEN] {orden}\n [COMANDO] {comando_lugar[indice_lugar]} {comando_accion[indice_accion]}\n </s>"

  return sentencia


# Nombre del archivo de texto en el que se guardarán las sentencias
nombre_archivo = "sentencias.txt"

contador_sentencias = 0

# Abre el archivo en modo escritura
with open(nombre_archivo, "w") as archivo:
    # Itera para crear y guardar las sentencias
    for _ in range(1000000):
        sentencia = crea_sentencia()
        archivo.write(sentencia + "\n")
        contador_sentencias += 1
        if contador_sentencias % 10000 == 0:
            print(f"Se han generado {contador_sentencias} sentencias.")

print(f"Se han creado y guardado {contador_sentencias} sentencias en el archivo:", nombre_archivo)



Entrenamos el modelo

In [ ]:
!./train-text-from-scratch --vocab-model ./models/ggml-vocab-llama.gguf --ctx 64 --embd 256 --head 16 --layer 32 --checkpoint-in chk-onoff-256x16x32.gguf --checkpoint-out chk-onoff-256x16x32.gguf --model-out ggml-onoff-256x16x32-f32.gguf --train-data "sentencias.txt" -b 64 --seed 1 --adam-iter 512 --no-checkpointing